In [3]:
import pandas as pd
import numpy as np
from scipy import sparse
import sys
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

### recommender code

In [4]:
all = pd.read_csv('../datasets/all.csv')
movies = pd.read_csv('../datasets/movies.csv')
pivot = all.pivot_table(index='title', columns=['userId'], values='rating')
        
pivot_sparse = sparse.csr_matrix(pivot.fillna(0))

distances = pairwise_distances(pivot_sparse, metric='cosine')
similarity = 1.0 - distances

recommend = pd.DataFrame(similarity, index=pivot.index, columns=pivot.index)

### bechdel score grabber function

In [5]:
def bechdel(title):
    return (all.loc[all['title'] == title]['bechdel_score']).mode()[0]

### query code borrowed heavily from [the recommender lecture]('https://git.generalassemb.ly/DSIR-0124/lesson-recommender-systems')

In [8]:
query = 'what about bob'

movie_list = []
for title in movies.loc[movies['title'].str.contains(query, case=False), 'title']:
    # movie_list.append(title)
    # movie_df = pd.DataFrame(movie_list, columns = ['movies']) #this allows dropdown in streamlit
    results = recommend[title].sort_values(ascending=False)[1:6]
    results_df = pd.DataFrame(results)
    results_df['bechdel score'] = results_df.index.map(bechdel)
    print(results_df)
    print()

                            What About Bob? (1991)  bechdel score
title                                                            
Better Off Dead... (1985)                 0.537840              3
Christmas Story, A (1983)                 0.471911              1
Wayne's World (1992)                      0.457453              1
Wedding Singer, The (1998)                0.436379              3
Labyrinth (1986)                          0.435760              3



### results with bechdel score

In [5]:
results_df

,Tokyo Godfathers (2003),bechdel score
title,,
Harakiri (Seppuku) (1962),0.613894,0
Millennium Actress (Sennen joyû) (2001),0.604975,2
"Day of the Beast, The (Día de la Bestia, El) (1995)",0.565233,3
Jersey Boys (2014),0.565233,2
Ocho apellidos vascos (2014),0.565233,1


In [6]:
all

,title,genres,imdbId,userId,rating,bechdel_score,year
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,1,4.0,1,1995.0
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,5,4.0,1,1995.0
2,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,7,4.5,1,1995.0
3,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,15,2.5,1,1995.0
4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,17,4.5,1,1995.0
...,...,...,...,...,...,...,...
82854,BlacKkKlansman (2018),Comedy|Crime|Drama,7349662,462,2.5,3,2018.0
82855,The Darkest Minds (2018),Sci-Fi|Thriller,4073790,50,3.5,3,2018.0
82856,John From (2015),Drama,3977428,338,1.0,3,2015.0
82857,Love Live! The School Idol Movie (2015),Animation,3837248,184,4.0,3,2015.0


In [11]:
pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
"'burbs, The (1989)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'night Mother (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(500) Days of Summer (2009),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5
10 Cloverfield Lane (2016),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [12]:
distances

array([[0.        , 1.        , 1.        , ..., 0.86056877, 1.        ,
        1.        ],
       [1.        , 0.        , 0.92857143, ..., 0.91036563, 0.62712421,
        1.        ],
       [1.        , 0.92857143, 0.        , ..., 1.        , 0.819991  ,
        1.        ],
       ...,
       [0.86056877, 0.91036563, 1.        , ..., 0.        , 0.89960449,
        1.        ],
       [1.        , 0.62712421, 0.819991  , ..., 0.89960449, 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        0.        ]])

In [13]:
similarity

array([[1.        , 0.        , 0.        , ..., 0.13943123, 0.        ,
        0.        ],
       [0.        , 1.        , 0.07142857, ..., 0.08963437, 0.37287579,
        0.        ],
       [0.        , 0.07142857, 1.        , ..., 0.        , 0.180009  ,
        0.        ],
       ...,
       [0.13943123, 0.08963437, 0.        , ..., 1.        , 0.10039551,
        0.        ],
       [0.        , 0.37287579, 0.180009  , ..., 0.10039551, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [14]:
recommend

title,'71 (2014),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),...,Zoom (2006),Zootopia (2016),Zulu (1964),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.000000,0.000000,0.000000,0.141653,0.285169,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.178042,0.000000,0.342055,0.543305,0.707107,0.000000,0.139431,0.000000,0.0
"'burbs, The (1989)",0.000000,1.000000,0.071429,0.099735,0.000000,0.180401,0.036417,0.125357,0.254537,0.000000,...,0.0,0.038152,0.025986,0.000000,0.000000,0.000000,0.211467,0.089634,0.372876,0.0
'night Mother (1986),0.000000,0.071429,1.000000,0.000000,0.000000,0.092513,0.000000,0.091168,0.171736,0.000000,...,0.0,0.000000,0.363803,0.000000,0.000000,0.000000,0.216295,0.000000,0.180009,0.0
(500) Days of Summer (2009),0.141653,0.099735,0.000000,1.000000,0.173843,0.326122,0.225255,0.206167,0.198959,0.093467,...,0.0,0.260308,0.000000,0.149685,0.076961,0.100164,0.097935,0.276512,0.169385,0.0
10 Cloverfield Lane (2016),0.285169,0.000000,0.000000,0.173843,1.000000,0.036935,0.133272,0.045497,0.022956,0.000000,...,0.0,0.341125,0.000000,0.097544,0.154934,0.201645,0.200462,0.121770,0.032083,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[REC]³ 3 Génesis (2012),0.707107,0.000000,0.000000,0.100164,0.201645,0.000000,0.000000,0.128932,0.091077,0.244949,...,0.0,0.283263,0.428746,0.544207,0.713468,1.000000,0.000000,0.246482,0.000000,0.0
eXistenZ (1999),0.000000,0.211467,0.216295,0.097935,0.200462,0.167085,0.114869,0.094899,0.130010,0.000000,...,0.0,0.086647,0.078689,0.000000,0.000000,0.000000,1.000000,0.192259,0.170341,0.0
xXx (2002),0.139431,0.089634,0.000000,0.276512,0.121770,0.176720,0.226588,0.198621,0.285099,0.112701,...,0.0,0.234282,0.202902,0.305535,0.173151,0.246482,0.192259,1.000000,0.100396,0.0
